# 한국거래소(KRX) - 전체종목 리스트 (종목검색기)

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd 
import requests
import json
import time

* Chrome() 혹은 PhantomJS() 실행 파일 하겠습니다. 
* 실제 위치한 경로를 적어줍니다. 

In [2]:
driver = webdriver.Chrome("C:\chromedriver\chromedriver.exe")
#driver = webdriver.PhantomJS(r"C:/phantomjs-2.1.1-windows/bin/phantomjs.exe")

In [3]:
url = "http://marketdata.krx.co.kr/mdi#document=040201"

* get() 메소드를 사용하여 드라이버에게 내가 이동하고 싶은 주소로 이동 요청을 합니다.
* sleep() 함수를 사용하여 페이지가 완전히 로딩될 때까지 5초간 기다립니다. 

In [4]:
driver.get(url)
time.sleep(5)

In [5]:
driver.find_elements_by_css_selector("#finderbtna87ff679a2f3e71d9181a67b7542122c")[0].click()
time.sleep(5)

* data-code 값을 post 방식의 requests 항목에 넣어야 합니다.
* 해당 값은 접속 할때마다 변경되며, 한번 값을 입력하고 사용할 수 없습니다. 
* 드라이버의 page_source는 화면에 보이는 웹의 소스코드를 출력해줍니다.
    > 만약 자바스크립트로 되어 있다고 해도 결과를 변환하여 출력합니다.
* data-code 값을 찾아서 post의 request의 데이터에 값을 입력합니다.

In [6]:
bs = BeautifulSoup(driver.page_source, "html.parser") 
rt2 = bs.find("form", {"action" : "/contents/MKD/99/MKD99000001.jspx", 
                       "method" : "post", "data-bld":"COM/finder_stkisu"})


payload = {'no'	: 'P1',
           'mktsel' : 'ALL',	
           'pagePath' :	'/contents/COM/FinderStkIsu.jsp'
           }

payload['code'] = rt2['data-code']

In [7]:
payload

{'no': 'P1',
 'mktsel': 'ALL',
 'pagePath': '/contents/COM/FinderStkIsu.jsp',
 'code': 'wwlgpe/TY12PWXEIMNwWgk3N1QzcmlbYwe4fDFBWiU57O32H9065wHsCOo9Ukq1eRIHZe8iNKJRZRfZP4tOMrkc77IwiDUPvyGumt+248Ey6jTbCNUL9YS2q8uAPQ7jX'}

In [8]:
rt_list = []
for cnt in range(1,16):
    payload['consonant'] = cnt
    rt3 = requests.post("http://marketdata.krx.co.kr/contents/MKD/99/MKD99000001.jspx",  data=payload)
    #print (rt3.text)
    rt_list.append(rt3.text)

* 결과를 하나만 확인해보겠습니다. 

In [9]:
rt_list[0]

'{"block1":[{"full_code":"KR7079940003","short_code":"A079940","codeName":"가비아","marketName":"KOSDAQ"},{"full_code":"KR7078890001","short_code":"A078890","codeName":"가온미디어","marketName":"KOSDAQ"},{"full_code":"KR7000500009","short_code":"A000500","codeName":"가온전선","marketName":"KOSPI"},{"full_code":"KR7192410009","short_code":"A192410","codeName":"감마누","marketName":"KOSDAQ"},{"full_code":"KR7000860007","short_code":"A000860","codeName":"강남제비스코","marketName":"KOSPI"},{"full_code":"KR7217730001","short_code":"A217730","codeName":"강스템바이오텍","marketName":"KOSDAQ"},{"full_code":"KR7035250000","short_code":"A035250","codeName":"강원랜드","marketName":"KOSPI"},{"full_code":"KR7011420007","short_code":"A011420","codeName":"갤럭시아에스엠","marketName":"KOSPI"},{"full_code":"KR7094480001","short_code":"A094480","codeName":"갤럭시아컴즈","marketName":"KOSDAQ"},{"full_code":"KR7063080006","short_code":"A063080","codeName":"게임빌","marketName":"KOSDAQ"},{"full_code":"KR7039240007","short_code":"A039240","codeName":"경

## json 파싱
* 결과 값이 dict 형태로 보이지만 문자열이기 때문에 json의 loads메소드를 사용하여 dict 형태로 변경합니다.
* total_df라는 빈 dataframe을 생성하고 concat 함수를 사용하여 계속 row 기준으로 dataframe을 생성하여 붙입니다.

In [10]:
total_df = pd.DataFrame()

In [11]:
total_df

""


In [12]:
for x in rt_list:
    rt = json.loads(x)
    # 데이터프레임 생성 후 concat로 total_df로 붙입니다. 
    total_df = pd.concat([total_df, pd.DataFrame(rt['block1'])])

In [13]:
total_df.shape

(2445, 4)

In [14]:
total_df.head(3)

,codeName,full_code,marketName,short_code
0,가비아,KR7079940003,KOSDAQ,A079940
1,가온미디어,KR7078890001,KOSDAQ,A078890
2,가온전선,KR7000500009,KOSPI,A000500


In [15]:
total_df.tail()

,codeName,full_code,marketName,short_code
230,W홀딩컴퍼니,KR7052300001,KOSDAQ,A052300
231,YG PLUS,KR7037270006,KOSPI,A037270
232,YTN,KR7040300006,KOSDAQ,A040300
233,YW,KR7051390003,KOSDAQ,A051390
234,iMBC,KR7052220001,KOSDAQ,A052220


In [16]:
total_df[total_df.codeName == "삼성전자"]

,codeName,full_code,marketName,short_code
29,삼성전자,KR7005930003,KOSPI,A005930


In [17]:
total_df.query('codeName == "삼성전자"')

,codeName,full_code,marketName,short_code
29,삼성전자,KR7005930003,KOSPI,A005930


* 삼성이 들어가는 종목 검색을 해봅시다.

In [18]:
total_df.query('codeName.str.contains("삼성")')

,codeName,full_code,marketName,short_code
16,삼성SDI,KR7006400006,KOSPI,A006400
17,삼성SDI우,KR7006401004,KOSPI,A006405
18,삼성공조,KR7006660005,KOSPI,A006660
19,삼성머스트스팩3호,KR7309930006,KOSDAQ,A309930
20,삼성물산,KR7028260008,KOSPI,A028260
21,삼성물산우B,KR702826K016,KOSPI,A02826K
22,삼성바이오로직스,KR7207940008,KOSPI,A207940
23,삼성생명,KR7032830002,KOSPI,A032830
24,삼성스팩2호,KR7291230001,KOSDAQ,A291230
25,삼성에스디에스,KR7018260000,KOSPI,A018260


In [19]:
total_df.reset_index(drop=True, inplace=True)

In [20]:
total_df

,codeName,full_code,marketName,short_code
0,가비아,KR7079940003,KOSDAQ,A079940
1,가온미디어,KR7078890001,KOSDAQ,A078890
2,가온전선,KR7000500009,KOSPI,A000500
3,감마누,KR7192410009,KOSDAQ,A192410
4,강남제비스코,KR7000860007,KOSPI,A000860
5,강스템바이오텍,KR7217730001,KOSDAQ,A217730
6,강원랜드,KR7035250000,KOSPI,A035250
7,갤럭시아에스엠,KR7011420007,KOSPI,A011420
8,갤럭시아컴즈,KR7094480001,KOSDAQ,A094480
9,게임빌,KR7063080006,KOSDAQ,A063080


## 과제 
#### DataFrame의 저장 메소드를 사용해서 csv, excel등으로 저장하세요.